In [6]:
# Import necessary libraries
import pandas as pd
import sqlite3

# Read the CSV file into a DataFrame
df = pd.read_csv("vgsales.csv")

# Display the first few rows to check the data
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Create a connection to a new SQLite database file
conn = sqlite3.connect("games.db")

# Save the DataFrame to a table named 'games'
df.to_sql("games", conn, if_exists="replace", index=False)

# Print confirmation
print("Data successfully loaded into SQLite database.")

Data successfully loaded into SQLite database.


In [10]:
# Check the structure of the 'games' table
query = "PRAGMA table_info(games);"
pd.read_sql_query(query, conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Name,TEXT,0,None,0
1,1,Platform,TEXT,0,None,0
2,2,Year_of_Release,REAL,0,None,0
3,3,Genre,TEXT,0,None,0
4,4,Publisher,TEXT,0,None,0
5,5,NA_Sales,REAL,0,None,0
6,6,EU_Sales,REAL,0,None,0
7,7,JP_Sales,REAL,0,None,0
8,8,Other_Sales,REAL,0,None,0
9,9,Global_Sales,REAL,0,None,0


In [12]:
# Select the first 5 rows from the table
query = "SELECT * FROM games LIMIT 5;"
pd.read_sql_query(query, conn)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,None,NaN,None,None
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,None,NaN,None,None


In [14]:
# Count how many rows are in the table
query = "SELECT COUNT(*) AS total_records FROM games;"
pd.read_sql_query(query, conn)

,total_records
0,16719


In [16]:
# Count how many NULL values are in each column
columns = df.columns
null_counts = []

for col in columns:
    query = f"SELECT COUNT(*) AS null_count FROM games WHERE {col} IS NULL;"
    count = pd.read_sql_query(query, conn)
    null_counts.append((col, count['null_count'][0]))

# Convert to DataFrame for display
pd.DataFrame(null_counts, columns=["Column", "Null_Count"])

,Column,Null_Count
0,Name,2
1,Platform,0
2,Year_of_Release,269
3,Genre,2
4,Publisher,54
5,NA_Sales,0
6,EU_Sales,0
7,JP_Sales,0
8,Other_Sales,0
9,Global_Sales,0


In [18]:
# View unique genres
query = "SELECT DISTINCT Genre FROM games;"
pd.read_sql_query(query, conn)

,Genre
0,Sports
1,Platform
2,Racing
3,Role-Playing
4,Puzzle
5,Misc
6,Shooter
7,Simulation
8,Action
9,Fighting


In [20]:
# View unique platforms
query = "SELECT DISTINCT Platform FROM games;"
pd.read_sql_query(query, conn)

,Platform
0,Wii
1,NES
2,GB
3,DS
4,X360
5,PS3
6,PS2
7,SNES
8,GBA
9,PS4


In [22]:
# View unique publishers
query = "SELECT DISTINCT Publisher FROM games;"
pd.read_sql_query(query, conn)

,Publisher
0,Nintendo
1,Microsoft Game Studios
2,Take-Two Interactive
3,Sony Computer Entertainment
4,Activision
...,...
577,Commseed
578,UIG Entertainment
579,Takuyo
580,Interchannel-Holon


# 🔍 Question 1: What is the average global sales by genre?

In [24]:
query = """
SELECT Genre, ROUND(AVG(Global_Sales), 2) AS Avg_Global_Sales
FROM games
WHERE Global_Sales IS NOT NULL
GROUP BY Genre
ORDER BY Avg_Global_Sales DESC;
"""

pd.read_sql_query(query, conn)

,Genre,Avg_Global_Sales
0,None,1.21
1,Platform,0.93
2,Shooter,0.80
3,Role-Playing,0.62
4,Racing,0.58
5,Sports,0.57
6,Fighting,0.53
7,Action,0.52
8,Misc,0.46
9,Simulation,0.45


# 🔍 Question 2: What are the yearly sales trends by platform?

In [26]:
query = """
SELECT Year_of_Release AS Year, Platform, ROUND(SUM(Global_Sales), 2) AS Total_Sales
FROM games
WHERE Year_of_Release IS NOT NULL AND Global_Sales IS NOT NULL
GROUP BY Year, Platform
ORDER BY Year ASC, Total_Sales DESC;
"""

pd.read_sql_query(query, conn)

,Year,Platform,Total_Sales
0,1980.0,2600,11.38
1,1981.0,2600,35.77
2,1982.0,2600,28.86
3,1983.0,NES,10.96
4,1983.0,2600,5.83
...,...,...,...
236,2016.0,X360,1.52
237,2016.0,Wii,0.18
238,2017.0,PS4,0.04
239,2017.0,PSV,0.02


# 🔍 Question 3: What is the average sales of the top 3 games by each publisher?

In [28]:
query = """
WITH RankedGames AS (
    SELECT 
        Publisher,
        Name,
        Global_Sales,
        RANK() OVER (PARTITION BY Publisher ORDER BY Global_Sales DESC) AS rank
    FROM games
    WHERE Publisher IS NOT NULL AND Global_Sales IS NOT NULL
)

SELECT 
    Publisher,
    ROUND(AVG(Global_Sales), 2) AS Avg_Top3_Sales
FROM RankedGames
WHERE rank <= 3
GROUP BY Publisher
ORDER BY Avg_Top3_Sales DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,Publisher,Avg_Top3_Sales
0,Nintendo,52.76
1,Take-Two Interactive,19.37
2,Activision,14.66
3,Microsoft Game Studios,14.60
4,Sony Computer Entertainment,12.53
5,Ubisoft,8.92
6,Electronic Arts,8.25
7,Bethesda Softworks,7.45
8,Sega,6.37
9,Konami Digital Entertainment,6.02


# 🔍 Question 4: What is the relationship between Critic Score and Global Sales?

In [30]:
query = """
SELECT 
    Name,
    Platform,
    Critic_Score,
    Global_Sales
FROM games
WHERE Critic_Score IS NOT NULL AND Global_Sales IS NOT NULL
ORDER BY Critic_Score DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,Name,Platform,Critic_Score,Global_Sales
0,Grand Theft Auto IV,X360,98.0,11.01
1,Grand Theft Auto IV,PS3,98.0,10.50
2,Tony Hawk's Pro Skater 2,PS,98.0,4.68
3,SoulCalibur,DC,98.0,0.34
4,Grand Theft Auto V,PS3,97.0,21.04
5,Grand Theft Auto V,X360,97.0,16.27
6,Grand Theft Auto III,PS2,97.0,13.10
7,Grand Theft Auto V,PS4,97.0,12.61
8,Super Mario Galaxy,Wii,97.0,11.35
9,Super Mario Galaxy 2,Wii,97.0,7.51


# 🔍 Question 5: Which game has the largest difference between User Score and Critic Score?

In [32]:
# Convert User_Score to numeric (set errors='coerce' to turn non-numeric values into NaN)
df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')

# Convert 10-point User Score to 100-point scale
df['User_Score'] = df['User_Score'] * 10

# Overwrite the existing table with cleaned data
df.to_sql("games", conn, if_exists="replace", index=False)

print("User_Score cleaned and table updated.")

User_Score cleaned and table updated.


In [34]:
query = """
SELECT 
    Name,
    Platform,
    Critic_Score,
    User_Score,
    ABS(Critic_Score - User_Score) AS Score_Difference
FROM games
WHERE Critic_Score IS NOT NULL AND User_Score IS NOT NULL
ORDER BY Score_Difference DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,Name,Platform,Critic_Score,User_Score,Score_Difference
0,Thrillville: Off the Rails,PSP,72.0,6.0,66.0
1,Shrek Swamp Kart Speedway,GBA,27.0,89.0,62.0
2,Company of Heroes 2,PC,80.0,21.0,59.0
3,Call of Duty: Modern Warfare 3,PS3,88.0,32.0,56.0
4,Call of Duty: Modern Warfare 3,X360,88.0,34.0,54.0
5,Call of Duty: Modern Warfare 3,PC,78.0,25.0,53.0
6,Thrillville: Off the Rails,PS2,70.0,17.0,53.0
7,Call of Duty: Modern Warfare 3,Wii,70.0,18.0,52.0
8,Antz Extreme Racing,XB,35.0,87.0,52.0
9,Football Manager Live,PC,82.0,31.0,51.0
